In [2]:
import pandas as pd

# Charger le fichier
df = pd.read_csv("./Sample.txt", delimiter="\t", skiprows=3, header=None)

# Remplacer les valeurs vides par NaN
df = df.replace("", float("nan"))

# Extraire la première colonne des colonnes 6 à 8 (donc colonne 6, index 5)
signal_data = df.iloc[:, 5]  

# Sauvegarder les valeurs dans un fichier temporaire
signal_data.to_csv("signal_data.txt", index=False, header=False)

print("✅ Données enregistrées dans signal_data.txt")

✅ Données enregistrées dans signal_data.txt


In [ ]:
import pandas as pd
import numpy as np

# Charger les données
file_path = "signal_data.txt"
data = pd.read_csv(file_path, header=None, names=["Acc_X"])

# Définition du centre dynamique
data["Moyenne_Centre"] = data["Acc_X"].rolling(window=10, center=True).mean()

# Calcul de la variation
data["Variation"] = data["Acc_X"].diff()

# Initialisation des variables
seuil_mouvement = 30  # Seuil pour détecter un vrai mouvement
cooldown_max = 10  # Nombre de points avant de réaccepter un nouveau mouvement
mouvement_actuel = "Stable"  # État actuel du mouvement
cooldown = 0  # Compteur pour éviter les inversions immédiates
data["Mouvement"] = "⚪ Stable"

# Détection des mouvements plus réaliste
for i in range(1, len(data)):
    valeur = data["Acc_X"].iloc[i]
    variation = data["Variation"].iloc[i]

    # Réduire le cooldown si on est dans une période de blocage
    if cooldown > 0:
        cooldown -= 1
        continue  # On saute cette itération pour ne pas détecter un nouveau mouvement

    # Détection d'une accélération vers la droite
    if valeur > 510 and variation > seuil_mouvement and mouvement_actuel == "Stable":
        mouvement_actuel = "Droite"
        data.loc[i, "Mouvement"] = "➡️ Droite"
        cooldown = cooldown_max  # Bloquer les détections inverses pendant X échantillons

    # Détection d'un retour à la position stable après un mouvement à droite
    elif mouvement_actuel == "Droite" and valeur < 510:
        mouvement_actuel = "Stable"

    # Détection d'une accélération vers la gauche
    elif valeur < 500 and variation < -seuil_mouvement and mouvement_actuel == "Stable":
        mouvement_actuel = "Gauche"
        data.loc[i, "Mouvement"] = "⬅️ Gauche"
        cooldown = cooldown_max  # Bloquer les détections inverses pendant X échantillons

    # Détection d'un retour à la position stable après un mouvement à gauche
    elif mouvement_actuel == "Gauche" and valeur > 500:
        mouvement_actuel = "Stable"

# Sauvegarde des résultats
output_file = "all_data_with_movement.csv"
data.to_csv(output_file, index=False)

print(f"✅ Détection des mouvements terminée et enregistrée dans {output_file}")

✅ Détection des mouvements terminée et enregistrée dans all_data_with_movement.csv


: 

In [ ]:
import tkinter as tk
import pandas as pd
import time
import socket
import threading

class BallApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Interface de Contrôle")
        self.root.geometry("800x600")
        self.root.resizable(False, False)
        
        # Création du Canvas principal
        self.canvas = tk.Canvas(root, bg="white", width=600, height=400, highlightthickness=5, highlightbackground="black")
        self.canvas.place(x=20, y=20)
        
        # Création de la boule rouge
        self.ball_size = 40  # Diamètre de la boule
        self.ball_x = 300
        self.ball_y = 200
        self.ball = self.canvas.create_oval(
            self.ball_x - self.ball_size / 2, self.ball_y - self.ball_size / 2,
            self.ball_x + self.ball_size / 2, self.ball_y + self.ball_size / 2,
            fill="red", outline="black"
        )
        
        # Panneau de contrôle sur la droite
        self.control_panel = tk.Frame(root, width=150, height=400, bg="white", highlightthickness=2, highlightbackground="black")
        self.control_panel.place(x=650, y=20)
        
        # Indicateurs "Actif" et "Clic"
        self.label_actif = tk.Label(self.control_panel, text="Actif", font=("Arial", 14))
        self.label_actif.pack(pady=(20, 5))
        self.canvas_actif = tk.Canvas(self.control_panel, width=30, height=30, bg="white", highlightthickness=1, highlightbackground="black")
        self.canvas_actif.pack()
        self.indicator_actif = self.canvas_actif.create_oval(5, 5, 25, 25, fill="red")
        
        self.label_clic = tk.Label(self.control_panel, text="Clic", font=("Arial", 14))
        self.label_clic.pack(pady=(20, 5))
        self.canvas_clic = tk.Canvas(self.control_panel, width=30, height=30, bg="white", highlightthickness=1, highlightbackground="black")
        self.canvas_clic.pack()
        self.indicator_clic = self.canvas_clic.create_oval(5, 5, 25, 25, fill="red")
        
        # Zone de log
        self.label_log = tk.Label(self.control_panel, text="Log", font=("Arial", 14))
        self.label_log.pack(pady=(20, 5))
        self.log_box = tk.Text(self.control_panel, height=6, width=18, state=tk.DISABLED)
        self.log_box.pack()
        
        # Configuration du serveur pour recevoir les données
        self.server_thread = threading.Thread(target=self.start_server, daemon=True)
        self.server_thread.start()

    def start_server(self):
        """Démarre un serveur pour recevoir des données en temps réel"""
        server = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server.bind(("0.0.0.0", 5002))
        server.listen(1)
        print("Serveur en attente de connexion...")
        
        conn, addr = server.accept()
        print(f"Connexion établie avec {addr}")
        
        while True:
            data = conn.recv(1024).decode("utf-8").strip()
            if not data:
                break
            self.process_data(data)

    def process_data(self, data):
        """Traite les données reçues et met à jour l'interface"""
        try:
            valeurs = data.split(",")  # Supposons que les données arrivent sous forme "Acc_X,Acc_Y"
            valeur_x = float(valeurs[0])
            valeur_y = float(valeurs[1])
            
            mouvement_x = "➡️ Droite" if valeur_x > 510 else "⬅️ Gauche" if valeur_x < 500 else "⚪ Stable"
            mouvement_y = "⬆️ Haut" if valeur_y > 510 else "⬇️ Bas" if valeur_y < 500 else "⚪ Stable"
            
            print(f"Mouvement détecté : {mouvement_x}, {mouvement_y}, Valeur X: {valeur_x}, Valeur Y: {valeur_y}")
            
            if mouvement_x == "➡️ Droite":
                deplacement_x = min(50, (valeur_x - 510) / 2)
                self.ball_x += deplacement_x
            elif mouvement_x == "⬅️ Gauche":
                deplacement_x = min(50, (500 - valeur_x) / 2)
                self.ball_x -= deplacement_x
            
            if mouvement_y == "⬆️ Haut":
                deplacement_y = min(50, (valeur_y - 510) / 2)
                self.ball_y -= deplacement_y  # Y diminue vers le haut
            elif mouvement_y == "⬇️ Bas":
                deplacement_y = min(50, (500 - valeur_y) / 2)
                self.ball_y += deplacement_y  # Y augmente vers le bas
            
            self.ball_x = max(self.ball_size, min(600 - self.ball_size, self.ball_x))
            self.ball_y = max(self.ball_size, min(400 - self.ball_size, self.ball_y))
            
            self.canvas.coords(
                self.ball,
                self.ball_x - self.ball_size / 2, self.ball_y - self.ball_size / 2,
                self.ball_x + self.ball_size / 2, self.ball_y + self.ball_size / 2
            )
        except ValueError:
            print("Erreur de conversion des données reçues.")

    def update_indicators(self, actif, clic):
        """Met à jour les couleurs des indicateurs Actif et Clic"""
        self.canvas_actif.itemconfig(self.indicator_actif, fill="green" if actif else "red")
        self.canvas_clic.itemconfig(self.indicator_clic, fill="green" if clic else "red")

if __name__ == "__main__":
    root = tk.Tk()
    app = BallApp(root)
    root.mainloop()


Serveur en attente de connexion...
Connexion établie avec ('127.0.0.1', 54466)
Mouvement détecté : ⬅️ Gauche, ⚪ Stable, Valeur X: 495.0, Valeur Y: 505.0
Mouvement détecté : ➡️ Droite, ⚪ Stable, Valeur X: 520.0, Valeur Y: 510.0
Mouvement détecté : ⚪ Stable, ⬇️ Bas, Valeur X: 510.0, Valeur Y: 490.0
Mouvement détecté : ⬅️ Gauche, ⬇️ Bas, Valeur X: 490.0, Valeur Y: 495.0
Mouvement détecté : ⚪ Stable, ⬆️ Haut, Valeur X: 505.0, Valeur Y: 520.0
Mouvement détecté : ➡️ Droite, ⚪ Stable, Valeur X: 515.0, Valeur Y: 500.0
Mouvement détecté : ⚪ Stable, ⬆️ Haut, Valeur X: 500.0, Valeur Y: 600.0
